# The following notebook shows the Flickr api GIS project created by Nathan Wies and Kazuto Nishimori
## The aim of this code is to find interesting trends in spatial Flickr data

In [2]:
"""
Code written by Nathan Wies and Kazuto Nishimori
1/11/20
"""

#Initialize api

import USERAUTH
import flickr_api
from flickr_api.api import flickr
import geopandas as gpd
import pandas as pd
import json

### Using the search method to pull data
#### Link to method documentation: https://www.flickr.com/services/api/flickr.photos.search.html

#### There is a pretty extensive list of spatial methods by which one can look up data which generally includes most parameters you could think of. All of these are posted at the link above.


##### bbox - 
A comma-delimited list of 4 values defining the Bounding Box of the area that will be searched.

The 4 values represent the bottom-left corner of the box and the top-right corner, minimum_longitude, minimum_latitude, maximum_longitude, maximum_latitude.

In [3]:
paris_pictures_json = flickr.photos.search(privacy_filter = 1, accuracy = 16, lat=35.6812006, 
                                lon=139.6887325, radius = 2, format= "json", nojsoncallback = 1)
parispics = json.loads(paris_pictures_json)
print("The total number of pictures found within 5 km of the center of Paris is",
    parispics["photos"]["total"])
parispics["photos"]["total"]


The total number of pictures found within 5 km of the center of Paris is 42629


'42629'

## Creating Dataframe from pulled data (parsing as json file)

In [4]:
"""
This method pulls data from the previous list. Unfortunatly this list is really long
so we NEED to check how many pings are acceptable or if there is a way to pull all of 
this data at once instead of sequentially, and if there is a limit to how much data 
we can pull per day.
"""
def create_df(pics):

    df = pd.DataFrame(columns =['id','latitude', 'longitude', 'accuracy', 'context', 'locality', 'county', 'region', 'country', 
                              'neighbourhood','title', "owner"])
    i=0
    for pic in pics["photos"]["photo"]:
        newfile = json.loads(flickr.photos.geo.getLocation(photo_id = pic["id"], format= "json", nojsoncallback = 1))
        df.loc[len(df)] = parse_json_data(newfile, pic["title"], pic["owner"])
        
        i+=1
        if(i>10):
            return df


def parse_json_data(output, title, owner):
    values = []
    output = output['photo']
    values.append(output['id'])
    output = output['location']
    values.append(output['latitude'])
    values.append(output['longitude'])
    values.append(output['accuracy'])
    values.append(output['context'])
    try:
        values.append(output['locality']['_content'])
    except:
        values.append("NaN")
    values.append(output['county']['_content'])
    values.append(output['region']['_content'])
    values.append(output['country']['_content'])
    values.append(output['neighbourhood']['_content'])
    values.append(title)
    values.append(owner)
    return values

df = create_df(parispics)
df      


,id,latitude,longitude,accuracy,context,locality,county,region,country,neighbourhood,title,owner
0,49374929351,48.864394,2.369233,16,0,Paris,Paris,Île-de-France,France,Paris 06 Ancien - Quartier Temple,48 Paris Janvier 2020 - boulevard Voltaire,7815453@N05
1,49374477173,48.864394,2.369233,16,0,Paris,Paris,Île-de-France,France,Paris 06 Ancien - Quartier Temple,47 Paris Janvier 2020 - boulevard Voltaire,7815453@N05
2,49374477043,48.865869,2.377675,16,0,Paris,Paris,Île-de-France,France,Paris 06 Ancien - Quartier Temple,49 Paris Janvier 2020 - la fresque de la rue O...,7815453@N05
3,49374435508,48.852937,2.353261,16,0,Paris,Paris,Île-de-France,France,Paris 09 Ancien,Visiting Paris. Four girls in the wind…,68047735@N00
4,49375072457,48.859147,2.267441,16,0,Paris,Paris,Île-de-France,France,Porte de passy,Au bord du lac (Morisot),7208148@N02
5,49371807757,48.871886,2.313863,16,0,Paris,Paris,Île-de-France,France,Paris 01 Ancien,Mon sac à moi,34645255@N06
6,49365969032,48.871658,2.314702,16,0,Paris,Paris,Île-de-France,France,Paris 01 Ancien,Echange furtif,34645255@N06
7,49374931837,48.860550,2.338049,16,0,Paris,Paris,Île-de-France,France,Paris 04 Ancien - Quartier Louvre,Au Louvre,71939658@N04
8,49374888812,48.846186,2.295177,16,0,Paris,Paris,Île-de-France,France,Grenelle,Paris,73422502@N08
9,49374493921,48.871666,2.378611,16,0,Paris,Paris,Île-de-France,France,Belleville,2020-01-12_07-57-57,66944824@N05


# Extra Stuff

In [5]:
#this method finds all photos associated with a user and attempts to find the geodata
def toymethod():
    user = flickr_api.Person.findByUserName("tomquirkphoto")
    photos = user.getPhotos()
    for photo in photos:
        print(flickr.photos.geo.getLocation(photo_id = photo["id"]))